# Emotet Static Config Extractor

Sample: `c7574aac7583a5bdc446f813b8e347a768a9f4af858404371eae82ad2d136a01`

Unpacked: `eeb13cd51faa7c23d9a40241d03beb239626fbf3efe1dbbfa3994fc10dea0827`

References:
- [Malshare Sample (Unpacked)](https://malshare.com/sample.php?action=detail&hash=eeb13cd51faa7c23d9a40241d03beb239626fbf3efe1dbbfa3994fc10dea0827)

## Helper Functions

In [1]:
def unhex(hex_string):
    import binascii
    if type(hex_string) == str:
        return binascii.unhexlify(hex_string.encode('utf-8'))
    else:
        return binascii.unhexlify(hex_string)

def tohex(data):
    import binascii
    if type(data) == str:
        return binascii.hexlify(data.encode('utf-8'))
    else:
        return binascii.hexlify(data)

## Load The Sample

In [3]:
import struct
import pefile

EMOTET_FILE = r'/tmp/emotet.bin'
data = open(EMOTET_FILE, 'rb').read()
pe = pefile.PE(data = data)


In [7]:
txt_data = None
for s in pe.sections:
    if b'.text' in s.Name:
        txt_data = s.get_data()
print(txt_data[:100])

b'Zy\xf8\x19\x12y\xf8\x19\x1f:\xab(zy\xf8\x19\x1a&\x8c\xaf\x9e\xa1$\x15gf\xfecm\xa5A\xe0\xed\xc4\xa6\x93u\xdfY\xebU\xd8\x81\x14N\x9c\r(\xb2\xc9\xf2\x07f\xf2\xc7b\xcad\xde?k\xff\x9de@\xd4!\xda\x06\xd5\xb0\xe9:\xfe\xe0\xc0.E\xa0\xe0\xe7fO@\xde\xfd\xee\xbd\x01\xd9\x15<\x00\x00\x00\x00\xc6R\xb2\x18'


## Decrypt Strings Table

In [12]:
def xor_decrypt(data, key):
    out = []
    for i in range(len(data)):
        out.append(data[i] ^ key[i%len(key)])
    return bytes(out)


def is_ascii(s):
    return all(c < 128 for c in s)

strings_table = []
ECS1_string = None
ECK1_string = None
for i in range(0,0x1000,4):
    candidate_1 = struct.unpack('<I',txt_data[i:i+4])[0] 
    candidate_2 = struct.unpack('<I',txt_data[i+4:i+8])[0] 
    if (candidate_1  & 0xffffff00) ^ (candidate_2 & 0xffffff00) == 0:
        # We have a match!
        key = txt_data[i:i+4]
        data_len = candidate_1 ^ candidate_2
        enc_data = txt_data[i+8:i+8+data_len]
        ptxt_data = xor_decrypt(enc_data, key)
        if is_ascii(ptxt_data):
            if ptxt_data != b'':
                strings_table.append(ptxt_data.decode('latin1'))
        if b'ECS1' == ptxt_data[:4]:
            ECS1_string = ptxt_data
        if b'ECK1' == ptxt_data[:4]:
            ECK1_string = ptxt_data

# Print our strings
print(ECS1_string)
print(ECK1_string)
for s in strings_table:
    print(s)


b'ECS1 \x00\x00\x00@_t\xb6\xc4\xd8\xdc\x0c=\x1f\x06z7\xdc\xb9\xf9\xb7\xbd^\x8a/\xa6\xa1\xf2\x0f\xa1y\r\x14\xe5\xf51\xe8\xb0\n\x1e<\x8b?{\x90\x1d&&1\x86e|\x1a\xad\xd9\xc3\\\xacH\xf0`\x87\x18\xd9t<X\xf9'
b'ECK1 \x00\x00\x00\xf3\xa35\xb5\x0e.+\xf45V\xcd\nL)>|\xf1\x10\xdd\xcb\xb0O \xb3\xfa\x02 \xceL\xb6\x0c\x1eD\x96\xbe\xb4\x0e\xe6\xc9[\x9a\xbdN\xbd\x9d\x8f\xcf\xe0\x10[4L\x82\x04&\x02\xd3\xba\xac\xf1\xfb\x9f,v'
%s\%s
%s\%s.exe
%s\*
%s:Zone.Identifier
POST
WinSta0\Default
%u.%u.%u.%u

--%S--
Cookie: %s=%s

Content-Type: multipart/form-data; boundary=%s


--%S
Content-Disposition: form-data; name="%S"; filename="%S"
Content-Type: application/octet-stream


shell32.dll
wtsapi32.dll
wininet.dll
crypt32.dll
advapi32.dll
urlmon.dll
shlwapi.dll
userenv.dll
bcrypt.dll
%s\%s
%s\rundll32.exe "%s\%s",%s
Control_RunDLL
SOFTWARE\Microsoft\Windows\CurrentVersion\Run
%s\%s%x
%s\rundll32.exe "%s\%s",%s %s
%s\rundll32.exe "%s",Control_RunDLL
%s%s.dll
%s%s.exe
%s\regsvr32.exe -s "%s"
DllRegisterServer
%s\ru

## Elliptic Curve Keys 

References:
- https://docs.microsoft.com/en-us/windows/win32/api/bcrypt/nf-bcrypt-bcryptimportkeypair
- https://docs.microsoft.com/en-us/windows/win32/api/bcrypt/ns-bcrypt-bcrypt_ecckey_blob
- https://github.com/tpn/winsdk-10/blob/master/Include/10.0.14393.0/shared/bcrypt.h
- https://pycryptodome.readthedocs.io/en/latest/src/public_key/ecc.html

In [25]:
print(ECS1_string)
print(ECK1_string)

# Extract ECS1 key
ECS1_key_len = struct.unpack('<I',ECS1_string[4:8])[0]
ECS1_x = int.from_bytes(ECS1_string[8:8+ECS1_key_len], "big")
ECS1_y = int.from_bytes(ECS1_string[8+ECS1_key_len:], "big")

from Crypto.PublicKey import ECC
ECS1_key = ECC.construct(curve="p256", point_x=ECS1_x, point_y=ECS1_y)
print("\nECS1 Key")
print(ECS1_key.export_key(format="PEM"))

# Extract ECK1 key
ECK1_key_len = struct.unpack('<I',ECK1_string[4:8])[0]
ECK1_x = int.from_bytes(ECK1_string[8:8+ECK1_key_len], "big")
ECK1_y = int.from_bytes(ECK1_string[8+ECK1_key_len:], "big")

from Crypto.PublicKey import ECC
ECK1_key = ECC.construct(curve="p256", point_x=ECK1_x, point_y=ECK1_y)
print("\nECK1 Key")
print(ECK1_key.export_key(format="PEM"))

b'ECS1 \x00\x00\x00@_t\xb6\xc4\xd8\xdc\x0c=\x1f\x06z7\xdc\xb9\xf9\xb7\xbd^\x8a/\xa6\xa1\xf2\x0f\xa1y\r\x14\xe5\xf51\xe8\xb0\n\x1e<\x8b?{\x90\x1d&&1\x86e|\x1a\xad\xd9\xc3\\\xacH\xf0`\x87\x18\xd9t<X\xf9'
b'ECK1 \x00\x00\x00\xf3\xa35\xb5\x0e.+\xf45V\xcd\nL)>|\xf1\x10\xdd\xcb\xb0O \xb3\xfa\x02 \xceL\xb6\x0c\x1eD\x96\xbe\xb4\x0e\xe6\xc9[\x9a\xbdN\xbd\x9d\x8f\xcf\xe0\x10[4L\x82\x04&\x02\xd3\xba\xac\xf1\xfb\x9f,v'

ECS1 Key
-----BEGIN PUBLIC KEY-----
MFkwEwYHKoZIzj0CAQYIKoZIzj0DAQcDQgAEQF90tsTY3Aw9HwZ6N9y5+be9Xoov
pqHyD6F5DRTl9THosAoePIs/e5AdJiYxhmV8Gq3Zw1ysSPBghxjZdDxY+Q==
-----END PUBLIC KEY-----

ECK1 Key
-----BEGIN PUBLIC KEY-----
MFkwEwYHKoZIzj0CAQYIKoZIzj0DAQcDQgAE86M1tQ4uK/Q1Vs0KTCk+fPEQ3cuw
TyCz+gIgzky2DB5Elr60DubJW5q9Tr2dj8/gEFs0TIIEJgLTuqzx+58sdg==
-----END PUBLIC KEY-----


## C2 Ground Truth

`81.0.236.93:443`

In [35]:
data_data = None
for s in pe.sections:
    if b'.data' in s.Name:
        data_data = s.get_data()
print(data_data[:100])

key = data_data[:4]
data_len = struct.unpack('<I',data_data[:4])[0] ^ struct.unpack('<I',data_data[4:8])[0]
enc_data = data_data[8:8+data_len]
ptxt_data = xor_decrypt(enc_data, key)

print(tohex(ptxt_data))

print("\n== C2 List== ")
for i in range(0,len(ptxt_data),8):
    print("%d.%d.%d.%d:%d" % (ptxt_data[i+0],ptxt_data[i+1],ptxt_data[i+2],ptxt_data[i+3],struct.unpack('>H',ptxt_data[i+4:i+6])[0]))



b'\xc7\x08\x945g\x08\x945\x96\x08xh\xc6\xb3\x944\x99\xb9lu\xc6\xb3\x944\x85"\xa30\xdc\xa0\x944\xa0\x00\x8eR\xd8\x98\x944~\xb0\x8d\xd8\xd8\x98\x944\xeaD$?\xd8\x98\x944{U\xe9A\xd8\x98\x944\xa0\x00\x8eS\xd8\x98\x944uG\x07w\xd8\x98\x944\xfd\xeb\xbe\xd9\xc7X\x944\xea~\x13\xfe\xdc\xa0\x944\xa0C]7'
b'5100ec5d01bb00015eb1f84001bb0001422a37051ba8000167081a671f900001b9b819ed1f9000012d4cb00a1f900001bc5d7d741f90000167081a661f900001b24f93421f9000013ae32aec005000012d7687cb1ba80001674bc90201bb0001c39a851401bb00012d8e72e71f900001d4ed05d101bb0001cf2654c31f90000168fbd62e1f9000018ab9481a1f9000013344af081f900001d239d9841f900001'

== C2 List== 
81.0.236.93:443
94.177.248.64:443
66.42.55.5:7080
103.8.26.103:8080
185.184.25.237:8080
45.76.176.10:8080
188.93.125.116:8080
103.8.26.102:8080
178.79.147.66:8080
58.227.42.236:80
45.118.135.203:7080
103.75.201.2:443
195.154.133.20:443
45.142.114.231:8080
212.237.5.209:443
207.38.84.195:8080
104.251.214.46:8080
138.185.72.26:8080
51.68.175.8:8080
210.57.217.132:8080